## <strong> 1. 미니 배치와 배치 크기 </strong>

다중 선형 회귀에서 사용한 데이터를 보자.

In [1]:
import torch

x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

전체 5개의 데이터를 하나의 행렬로 선언하여 전체 데이터에 대해 경사 하강법을 수행하여 학습했다. <br>
하지만 데이터가 커졌을 때 위와 같이 전체 데이터에 대해 경사 하강법을 수행하면 너무 많은 계산량이 필요하다. <br>

따라서 전체 데이터를 작은 단위로 나눠서 학습해야 하는데 이 단위를 **미니 배치(Mini Batch)**라고 한다. <br>

미니 배치 학습에서는 미니 배치의 개수만큼 경사 하강법을 수행해야 전체 데이터가 전부 사용되어 1 Epoch가 된다. <br>
미니 배치의 개수는 미니 배치의 크기에 따라 달라지는데 이때 미니 배치의 크기를 **batch size**라고 부른다.

* 배치 크기는 **보통 2의 제곱수를 사용한다.** <br> 
cpu와 gpu의 메모리가 2의 배수이므로 배치 크기가 2의 제곱수일 경우 데이터 송수신의 효율을 높일 수 있기 때문이다.

* 미니 배치 경사 하강법은 전체 데이터의 일부를 보고 수행하므로 **최적값 수렴 과정에서 조금 헤매기도 하지만 훈련 속도가 빠르다.**

## <strong> 2. 이터레이션 </strong>

**이터레이션(Iteration)**이란 한 번의 에포크 내에서 이루어지는 매개변수 가중치 W와 b의 업데이터 횟수이다. <br>
전체 데이터가 2,000일 때 batch size가 200이라면 총 이터레이션은 10개이다.

## <strong> 3. 데이터 로드하기 </strong>

파이토치에서는 데이터를 좀 더 쉽게 다룰 수 있도록 **데이터셋(Dataset)**과 **데이터로더(DataLoader)**를 제공한다. <br>
이를 사용하면 미니 배치 학습, 데이터 셔플, 병렬 처리 등을 간단히 수행할 수 있다. <br>

기본적인 사용법은 **Dataset을 정의하고, 이를 DataLoader에 전달해야 한다.** <br>
Dataset을 커스텀하여 만들 수도 있지만 여기서는 텐서를 입력받아 Dataset의 형태로 변환해주는 TensorDataset을 사용해보자.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

TensorDataset은 기본적으로 텐서를 입력 받는다.

In [4]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

dataset = TensorDataset(x_train, y_train)

데이터셋을 만들었다면 데이터로더를 사용할 수 있다. <br>
데이터로더는 기본적으로 **데이터셋, 미니 배치의 크기를 입력 받는다.** <br>
또한 Epoch마다 데이터셋을 섞어서 학습되는 순서를 바꾸려면 **shuffle=True**를 인자로 전달하면 된다.

In [19]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
for i in list(dataloader):
  print(i)

[tensor([[93., 88., 93.],
        [89., 91., 90.]]), tensor([[185.],
        [180.]])]
[tensor([[ 73.,  80.,  75.],
        [ 96.,  98., 100.]]), tensor([[152.],
        [196.]])]
[tensor([[73., 66., 70.]]), tensor([[142.]])]


모델과 옵티마이저를 선언하자.

In [20]:
model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

미니 배치 경사 하강법을 이용해 학습시키자.

In [23]:
nb_epochs = 2000
for epoch in range(nb_epochs+1):
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%500 == 0:
      print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        )) 

Epoch    0/2000 Batch 1/3 Cost: 3.062688
Epoch    0/2000 Batch 2/3 Cost: 6.417829
Epoch    0/2000 Batch 3/3 Cost: 8.883194
Epoch  500/2000 Batch 1/3 Cost: 2.795532
Epoch  500/2000 Batch 2/3 Cost: 0.970570
Epoch  500/2000 Batch 3/3 Cost: 4.740135
Epoch 1000/2000 Batch 1/3 Cost: 1.298281
Epoch 1000/2000 Batch 2/3 Cost: 1.023259
Epoch 1000/2000 Batch 3/3 Cost: 0.625995
Epoch 1500/2000 Batch 1/3 Cost: 0.321468
Epoch 1500/2000 Batch 2/3 Cost: 1.140725
Epoch 1500/2000 Batch 3/3 Cost: 1.080793
Epoch 2000/2000 Batch 1/3 Cost: 0.153534
Epoch 2000/2000 Batch 2/3 Cost: 0.417696
Epoch 2000/2000 Batch 3/3 Cost: 0.742096
